## DA

In [29]:
import nlpcda
from nlpcda.tools.Basetool import Basetool
import jieba

In [37]:
class WordPositionExchange(Basetool):
    '''随机词语交换。'''
    
    from nlpcda.tools.Basetool import Basetool
    import jieba
    
    def __init__(self, create_num: int = 5, change_rate: float = 0.05, char_gram: int = 3, seed: int = 1):
        super(WordPositionExchange, self).__init__('', create_num, change_rate, seed)
        self.char_gram = char_gram

    def __replace_one(self, one_sentence: str):
        # 变为字 数组
        # sen_chars = list(one_sentence)
        sen_chars = list(jieba.cut(one_sentence, cut_all=False))
        for i in range(len(sen_chars)):
            if self.random.random() < self.change_rate:
                # 非中文字不动！
                if self.__is_chinese(sen_chars[i]) == False:
                    continue
                # 交换位置
                change_i = self.__cpt_exchange_position(sen_chars, i)
                # 进行交换
                sen_chars[i], sen_chars[change_i] = sen_chars[change_i], sen_chars[i]
        return ''.join(sen_chars)

    def __cpt_exchange_position(self, sen_chars: list, position_i):
        # 计算出交换位置
        i = position_i
        j = position_i
        # 从position_i左边，找到第一个不是中文的位置，or 全是中文则不能超过char_gram范围
        while i > 0 and self.__is_chinese(sen_chars[i]) and abs(i - position_i) < self.char_gram:
            i -= 1
        # 从position_i右边，找到第一个不是中文的位置，or 全是中文则不能超过char_gram范围
        while j < len(sen_chars) - 1 and self.__is_chinese(sen_chars[j]) and abs(j - position_i) < self.char_gram:
            j += 1
        # 不是中文导致的推出，需要撤回位置
        if not self.__is_chinese(sen_chars[i]):
            if i < position_i:
                i += 1
        if not self.__is_chinese(sen_chars[j]):
            if j > position_i:
                j -= 1
        return self.random.randint(i, j)

    def __is_chinese(self, a_chr):
        return u'\u4e00' <= a_chr <= u'\u9fff'

    def replace(self, replace_str: str):
        replace_str = replace_str.replace('\n', '').strip()
        sentences = [replace_str]
        t = 0

        while len(sentences) < self.create_num:
            t += 1
            a_sentence = self.__replace_one(replace_str)

            if a_sentence not in sentences:
                sentences.append(a_sentence)
            if t > self.create_num * self.loop_t / self.change_rate:
                break
        return sentences

In [42]:
exc = WordPositionExchange(char_gram=5)

x = '能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现中华民族伟大复兴的重要保证。\n江苏卫视《一站到底》节目首次引入的人工智能答题机器人“汪仔”，是由搜狐历时9个月，投资约4000多万元打造而成的。'
exc.replace(x)

['能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现中华民族伟大复兴的重要保证。江苏卫视《一站到底》节目首次引入的人工智能答题机器人“汪仔”，是由搜狐历时9个月，投资约4000多万元打造而成的。',
 '能否解决腐败问题，能否控制人口增长，隔阂消除两岸能否，都是实现中华民族保证的复兴重要伟大。江苏卫视《一站到底》节目首次引入的人工智能答题机器人“汪仔”，是由搜狐历时9个月，投资约4000多万元打造而成的。',
 '能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现复兴伟大中华民族的重要保证。江苏卫视《一站到底》答题首次引入的人工智能节目机器人“汪仔”，是由搜狐历时9个月，投资约4000多万元打造而成的。',
 '能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现中华民族伟大复兴的重要保证。江苏卫视《到底一站》节目首次引入的人工智能答题机器人“汪仔”，历时由搜狐是9个月，投资约4000多万元打造而成的。',
 '能否解决腐败问题，能否控制人口增长，能否消除两岸隔阂，都是实现中华民族伟大复兴的重要保证。江苏卫视《一站到底》节目的引入首次人工智能答题机器人“汪仔”，是由搜狐历时9个月，约投资4000多万元打造而成的。']

## Easy Ensemble

In [67]:
import numpy as np
import pandas as pd

data = pd.read_csv('../data/data-org/train.csv', sep='\t').set_index('id')
minority_idx = np.argwhere((data.label == 0).values).flatten()

L = len(data)
k = 10

In [80]:
def _easy_ensenble_generate_kfolds(L, k, minority_idx, fold_size=None):
    s_fold = fold_size if fold_size else L // k 
    majority_idx = np.array(list(set(range(L)) - set(minority_idx)))
    minor_samples = np.random.choice(minority_idx, size=(k, s_fold//2), replace=True)
    major_samples = np.random.choice(majority_idx, size=(k, s_fold//2), replace=False)
    folds = np.concatenate((minor_samples, major_samples), axis=1)
    folds = folds[:, np.random.permutation(range(s_fold))]
    print(f'Using easy ensemble - each fold has {minor_samples.shape[1]} negatives in the training set, paired with {major_samples.shape[1]} positives')
    return folds

In [81]:
folds = _easy_ensenble_generate_kfolds(L, k, minority_idx)

Using easy ensemble - each fold has 2262 negatives in the training set, paired with 2262 positives


In [82]:
data

,label,text
id,,
1,1,通过大力发展社区教育，使我省全民终身学习的教育体系已深入人心。
2,1,再次投入巨资的英超劲旅曼城队能否在2010-2011年度的英超联赛中夺得英超冠军，曼联、切尔...
3,1,广西居民纸质图书的阅读率偏低，手机阅读将成为了广西居民极倾向的阅读方式。
4,1,文字书写时代即将结束，预示着人与字之间最亲密的一种关系已经终结。与此同时，屏幕文化造就了另一...
5,1,安徽合力公司2006年叉车销售强劲，销售收入涨幅很有可能将超过40%以上。公司预计2006年...
...,...,...
45244,0,进入5月以来，全国新增人感染H7N9禽流感病例呈明显下降趋势。
45245,1,建设中国新一代天气雷达监测网，能够明显改善对热带气旋或台风登陆位置及强度预报的准确性，尤其对...
45246,1,每当回忆起和他朝夕相处的一段生活，他那循循善诱的教导和那和蔼可亲的音容笑貌，又重新出现在我的面前。


In [77]:
fold0 = data.iloc[folds[0]].drop_duplicates()
len(fold0)

4320

In [79]:
len(fold0[fold0.label==0]) / 4320

0.47638888888888886